In [1]:
# Install MLflow
!pip install mlflow -q

# Import libraries
import os
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import average_precision_score
from google.colab import drive


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.7/648.7 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.9 MB/s eta 0:00:00


In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the dataset
data_path = "/content/drive/MyDrive/dvc_project/data"


Mounted at /content/drive


In [7]:
print("Loading data...")

from sklearn.feature_extraction.text import TfidfVectorizer

# Load data
train = pd.read_csv(os.path.join(data_path, "train.csv"))
valid = pd.read_csv(os.path.join(data_path, "validation.csv"))

# Separate features and target
X_train_raw, y_train = train["text"], train["label"]
X_valid_raw, y_valid = valid["text"], valid["label"]

# Convert text into numerical vectors using TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train_raw)
X_valid = vectorizer.transform(X_valid_raw)

print("Text data successfully vectorized!")

print("Data loaded successfully!")


Loading data...
Text data successfully vectorized!
Data loaded successfully!


In [8]:
# Set up an experiment in MLflow
mlflow.set_experiment("Spam_Classification")


<Experiment: artifact_location='file:///content/mlruns/595921464032175323', creation_time=1741160809907, experiment_id='595921464032175323', last_update_time=1741160809907, lifecycle_stage='active', name='Spam_Classification', tags={}>

In [9]:
def train_and_log_model(model, model_name):
    """
    Trains a model, logs it to MLflow, and prints AUCPR score.
    """
    with mlflow.start_run():
        # Train model
        model.fit(X_train, y_train)

        # Predict probabilities
        y_pred = model.predict_proba(X_valid)[:, 1]

        # Compute AUCPR score
        aucpr = average_precision_score(y_valid, y_pred)

        # Log AUCPR metric
        mlflow.log_metric("AUCPR", aucpr)

        # Log model
        mlflow.sklearn.log_model(model, model_name)

        print(f"{model_name} AUCPR: {aucpr}")


In [10]:
# Define models
models = {
    "RandomForest": RandomForestClassifier(n_estimators=100),
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "SVM": SVC(probability=True)
}

# Train and log each model
for name, model in models.items():
    train_and_log_model(model, name)


2025/03/05 07:48:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest AUCPR: 0.9837029179613737


2025/03/05 07:49:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


LogisticRegression AUCPR: 0.972349844151224


2025/03/05 07:49:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


SVM AUCPR: 0.9807715057456959
